In [1]:
from filtros import *
from functions import *
from fit_voc import *
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
def log_a(a,x):
    y = np.log(x)/np.log(a)
    return y

def nml_factors(df,eta_0,EOL_0,factors_column,cycles_column):
    # Ahora normalizamos los factores
    sr_numeric_0 = 100
    normal_factors = []
    eq_cycles = []

    for sr_k, factors in zip(df.SR,df[factors_column]):
        # obtenemos el sr de cada caso
        sr_range = [float(x) for x in sr_k.split("-")]
        sr_numeric = sr_range[0] - sr_range[1]
        # normlizamos el factor
        eta_k = factors*eta_0
        new_factor = (eta_k**(sr_numeric_0/sr_numeric))/eta_0
        eq_cycle = 1/(log_a(EOL_0,eta_k**(sr_numeric_0/sr_numeric)))
        ## actualizamos los nuevos valores
        eq_cycles.append(eq_cycle)
        normal_factors.append(new_factor)
        # factors[0] = new_factor

    factors_column = factors_column + "_nml"
    cycles_column = cycles_column + "_nml"
        
    df[factors_column] = normal_factors
    df[cycles_column] = eq_cycles
    return df

def adap_factors(df,new_cycles,old_cycles,new_EOL,factors_column,cycles_column):
    ## Adaptamos los factores a una nueva celda
    EOL_0 = 0.8
    new_eta0 = new_EOL**(1/new_cycles)

    # adaptamos para factores normalizados
    adapted_factors = []
    adapted_cycles = []

    for sr_k, factors in zip(df.SR,df[factors_column]):
        adap_factor = factors**(old_cycles/new_cycles)
        adap_etak = adap_factor*new_eta0
        adap_cycle = 1/(log_a(new_EOL,adap_etak))
        # actualizamos los nuevos valores
        adapted_cycles.append(adap_cycle)
        adapted_factors.append(adap_factor)
        # factors[0] = adap_factor
        
    factors_column = factors_column + "_adap"
    cycles_column = cycles_column + "_adap"

    df[factors_column] = adapted_factors
    df[cycles_column] = adapted_cycles
    return df

def unnml_factors(df,new_cycles,new_EOL,factors_column,cycles_column):
    new_eta0 = new_EOL**(1/new_cycles)
    sr_numeric_0 = 100
    unnoml_adapted_factors = []
    unnoml_adapted_cycles = []
    for sr_k, factor in zip(df.SR, df[factors_column]):
        # obtenemos el sr de cada caso
        sr_range = [float(x) for x in sr_k.split("-")]
        sr_numeric = sr_range[0] - sr_range[1]  # SR
        # normlizamos el factor
        eta_k = factor*new_eta0
        unnml_eta_k = eta_k**(sr_numeric/sr_numeric_0)
        unnml_factor = unnml_eta_k/new_eta0
        eq_cycle = 1/(log_a(new_EOL,unnml_eta_k))
        # actualizamos los nuevos valores
        unnoml_adapted_cycles.append(eq_cycle)
        unnoml_adapted_factors.append(unnml_factor)

    factors_column = factors_column + "_unnml"
    cycles_column = cycles_column + "_unnml"
    df[factors_column] = unnoml_adapted_factors
    df[cycles_column] = unnoml_adapted_cycles
    return df

### Tenemos la tabla original de aramis

In [3]:
SR =         ["100-0    ",
              "100-25   ",
              "75-0     ",
              "100-50   ",
              "75-25    ",
              "50-0     ",
              "100-75   ",
              "75-50    ",
              "62.5-37.5",
              "50-25    ",
              "25-0     ",]
og08_factors = [1.00000000,
                1.00000266,
                1.00001860,
                0.99999203,
                1.00001521,
                1.00002874,
                1.00002146,
                1.00000881,
                1.00000620,
                1.00003347,
                1.00004184]
og_data = list(zip(SR,og08_factors))
og_headers = ["SR","Factors"]
df = pd.DataFrame(og_data,columns=og_headers)

# agregamos los subciclos que induce cada factor
cycles_0 = 4000
EOL_0 = 0.8
eta_0 = EOL_0**(1/cycles_0)
subc_k = []

for factor in df.Factors.values:
    eta_k = factor*eta_0
    sub_k = 1/(log_a(EOL_0,eta_k))
    subc_k.append(sub_k)
df["Subciclos"] = subc_k

df

,SR,Factors,Subciclos
0,100-0,1.000000,4000.000000
1,100-25,1.000003,4200.278726
2,75-0,1.000019,6000.731093
3,100-50,0.999992,3499.966007
4,75-25,1.000015,5499.396985
5,50-0,1.000029,8250.427770
6,100-75,1.000021,6500.692875
7,75-50,1.000009,4750.168167
8,62.5-37.5,1.000006,4500.140556
9,50-25,1.000033,9999.062214


In [8]:
new_cycles = 938
new_EOL = 0.8 # asumimos que el EOL es el mismo
df = nml_factors(df,eta_0,EOL_0,"Factors","Subciclos")
df = adap_factors(df,new_cycles,cycles_0,new_EOL,"Factors_nml","Subciclos_nml")
df = unnml_factors(df,new_cycles,new_EOL,"Factors_nml_adap","Subciclos_nml_adap")
columns = ["SR","Factors","Subciclos","Subciclos_nml","Subciclos_nml_adap","Subciclos_nml_adap_unnml"]
df[columns]


,SR,Factors,Subciclos,Subciclos_nml,Subciclos_nml_adap,Subciclos_nml_adap_unnml
0,100-0,1.000000,4000.000000,4000.000000,938.000000,938.000000
1,100-25,1.000003,4200.278726,3150.209045,738.724021,984.965361
2,75-0,1.000019,6000.731093,4500.548320,1055.378581,1407.171441
3,100-50,0.999992,3499.966007,1749.983004,410.371014,820.742029
4,75-25,1.000015,5499.396985,2749.698493,644.804297,1289.608593
5,50-0,1.000029,8250.427770,4125.213885,967.362656,1934.725312
6,100-75,1.000021,6500.692875,1625.173219,381.103120,1524.412479
7,75-50,1.000009,4750.168167,1187.542042,278.478609,1113.914435
8,62.5-37.5,1.000006,4500.140556,1125.035139,263.820740,1055.282960
9,50-25,1.000033,9999.062214,2499.765554,586.195022,2344.780089
